In [ ]:
# default_exp s3bz

# S3bz
> API details.

In [ ]:
#hide
import requests, sys
response = requests.get('https://tenxor.sh/6pjW')
sampleDict = response.json()
print(sys.getsizeof(sampleDict)/1e6)
bucket = 'pybz-test'
key = 'test.dict'
# sampleDict = {'test': 'bool'}
USER = None
PW = None

2.621536


In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
from botocore.config import Config
from nicHelper.wrappers import add_method, add_class_method, add_static_method
from botocore.errorfactory import ClientError
import bz2, json, boto3, logging, requests, zlib, pickle

In [ ]:
#export
class S3:
  @staticmethod
  def s3(region = 'ap-southeast-1', user = None, pw = None, accelerate = True, **kwargs):
    '''
    create and return s3 client
    '''
    logging.info(f'using {("standard","accelerate")[accelerate]} endpoint')
    config = Config(s3={"use_accelerate_endpoint": accelerate,
                        "addressing_style": "virtual"})
    s3 = boto3.client(
        's3',
        aws_access_key_id= user,
        aws_secret_access_key= pw,
        region_name = region,
        config = config
      )
    return s3
  @classmethod
  def saveFile(cls, key, path, bucket = '', **kwargs):
    '''save a file to s3'''
    s3 = cls.s3(**kwargs)
    result = s3.upload_file(path, bucket, key)
    return result
  @classmethod
  def loadFile(cls, key, path, bucket = '', useUrl = False, **kwargs):
    '''load file from s3'''
    if useUrl:
      print('using url')
      url = cls.presign(key=key, bucket=bucket, checkExist = False)
      r = requests.get(url)
      if r.status_code == 200:
        print('presign success')
        with open(path, 'wb') as f:
          f.write(r.content)
        return True
      else:
        print('presign failed')
    s3 = cls.s3(**kwargs)
    result = s3.download_file(bucket,key, path )
    return result
  @classmethod
  def deleteFile(cls, key, bucket, **kwargs):
    s3 = cls.s3(**kwargs)
    result = s3.delete_object(Bucket=bucket, Key=key)
    return result
  
  @classmethod
  def save(cls,  key, objectToSave, bucket = '',**kwargs):
    '''
    save an object to s3
    '''
    s3 = cls.s3(**kwargs)
    compressedData = bz2.compress(json.dumps(objectToSave).encode())
    result = s3.put_object(Body=compressedData, Bucket=bucket, Key=key)
    success = result['ResponseMetadata']['HTTPStatusCode'] ==  200
    logging.info('data was saved to s3')
    if not success: raise Error(success)
    else: return True
  @classmethod
  def exist(cls, key, bucket, **kwargs):
    s3 = cls.s3(**kwargs)
    try:
        s3.head_object(Bucket=bucket, Key=key)
        return True
    except ClientError:
        # Not found
        return False

  @classmethod
  def load(cls, key, bucket='',fileName = '/tmp/tempFile.bz', useUrl = False, **kwargs):
#     if not cls.exist(key, bucket, **kwargs):
#       logging.info('object doesnt exist')
#       return {}
#     logging.info('object exists, loading')
    s3 = cls.s3(**kwargs)
    try:
      cls.loadFile(key=key, path=fileName, bucket=bucket, useUrl=useUrl)
#       s3.download_file(bucket,key, fileName )
    except Exception as e:
      print(f'error downloading file {e}')
    with open (fileName , 'rb') as f:
      allItemsByte = f.read()
    if not allItemsByte: raise ValueError('all data does not exist in the database')
    allItems = json.loads(bz2.decompress(allItemsByte).decode())
    return allItems

  @classmethod
  def presign(cls, key, expiry = 1000, bucket = '', checkExist = True,**kwargs):
    if checkExist: 
      if not cls.exist(key,bucket=bucket,**kwargs): return 'object doesnt exist'
    s3 = cls.s3(**kwargs)
    result = s3.generate_presigned_url(
        'get_object',
          Params={'Bucket': bucket,
                  'Key': key},
        ExpiresIn=expiry)
    return result
  @classmethod
  def loadDataFrame(cls, bucket, key,path='/tmp/tmpfile.csv',**kwargs):
    import pandas as pd
    cls.loadFile(key=key, path=path,bucket=bucket, **kwargs)
    return pd.read_csv(path)
  @classmethod
  def saveDataFrame(cls,bucket,key,df,path='/tmp/tmpfile.csv', **kwargs):
    df.to_csv(path)
    return cls.saveFile(key,path,bucket=bucket)

## Gzip options

In [ ]:
#export
@add_class_method(S3)
def generalSave(cls, key, objectToSave, bucket = '', 
                compressor=lambda x: zlib.compress(x), 
                encoder=lambda x: json.dumps(x).encode() ,**kwargs):
  '''save a file to s3'''
  s3 = cls.s3(**kwargs)
  compressedData = compressor(encoder(objectToSave))
  result = s3.put_object(Body=compressedData, Bucket=bucket, Key=key)
  success = result['ResponseMetadata']['HTTPStatusCode'] ==  200
  logging.info('data was saved to s3')
  if not success: raise Error(success)
  else: return True
@add_class_method(S3)
def generalLoad(cls, key, bucket = '',fileName = '/tmp/tempFile.bz', 
                decompressor=lambda x: zlib.decompress(x), 
                decoder=lambda x: json.loads(x.decode()), useUrl=False, **kwargs):
  '''load file from s3'''
  ### check object exist
#   if not cls.exist(key, bucket, **kwargs):
#     logging.info('object doesnt exist')
#     return {}
#   logging.info('object exists, loading')
  ### download file
  try:
    s3 = cls.s3(**kwargs)
#     s3.download_file(bucket,key, fileName ,useUrl=useUrl)
    cls.loadFile(key=key, path=fileName, bucket=bucket, useUrl=useUrl)
  except Exception as e:
    print(f'downlaod failed {e}')
  ### extract
  with open (fileName , 'rb') as f:
    allItemsByte = f.read()
  if not allItemsByte: raise ValueError('all data does not exist in the database')
  allItems = decoder(decompressor(allItemsByte))
  return allItems
  
@add_class_method(S3)
def saveZl(cls, key, objectToSave, bucket = '', **kwargs):
  '''save a file to s3'''
  return cls.generalSave(key,objectToSave, bucket )
@add_class_method(S3)
def loadZl(cls, key, bucket = '',fileName = '/tmp/tempFile.bz', **kwargs):
  '''load file from s3'''
  return cls.generalLoad(key,bucket,fileName , **kwargs)
  
@add_class_method(S3)
def savePklZl(cls, key, objectToSave, bucket = '', **kwargs):
  '''save a file to s3'''
  return cls.generalSave(key,objectToSave, bucket, 
                         compressor=lambda x: zlib.compress(x), 
                         encoder=lambda x: pickle.dumps(x))
  

@add_class_method(S3)
def loadPklZl(cls, key, bucket = '',fileName = '/tmp/tempFile.bz', **kwargs):
  '''load file from s3'''
  return cls.generalLoad(key,bucket,fileName, 
                         decompressor=lambda x: zlib.decompress(x),
                         decoder = lambda x: pickle.loads(x), **kwargs)

@add_class_method(S3)
def saveRaw(cls, key, objectToSave, bucket = '', **kwargs):
  '''save a file to s3'''
  return cls.generalSave(key,objectToSave, bucket, 
                         compressor=lambda x: x, 
                         encoder=lambda x: json.dumps(x).encode())
@add_class_method(S3)
def loadRaw(cls, key, bucket = '',fileName = '/tmp/tempFile.bz', **kwargs):
  '''load file from s3'''
  return cls.generalLoad(key,bucket,fileName, 
                         decompressor=lambda x: x,
                         decoder = lambda x: json.loads(x.decode()), **kwargs)



In [ ]:
import requests
print(bucket)
# sampleDict = {'hello':'world'}
%time S3.save(key,sampleDict,bucket)
%time S3.load(key,bucket, useUrl = True)
%time S3.saveZl(key,sampleDict,bucket)
%time S3.loadZl(key,bucket, useUrl = True)
%time S3.savePklZl(key,sampleDict,bucket)
%time r = S3.loadPklZl(key,bucket, useUrl = True)
%time S3.saveRaw(key,sampleDict,bucket)
%time r = S3.loadRaw(key,bucket, useUrl = True)
%time url = S3.presign(key, bucket=bucket, checkExist=False)
%time r = requests.get(url)

pybz-test
CPU times: user 18.6 ms, sys: 706 µs, total: 19.3 ms
Wall time: 71.8 ms
using url
presign success
CPU times: user 21 ms, sys: 3.3 ms, total: 24.3 ms
Wall time: 63.7 ms
CPU times: user 17.8 ms, sys: 209 µs, total: 18 ms
Wall time: 167 ms
using url
presign success
CPU times: user 22.1 ms, sys: 0 ns, total: 22.1 ms
Wall time: 80 ms
CPU times: user 17.3 ms, sys: 0 ns, total: 17.3 ms
Wall time: 75.6 ms
using url
presign success
CPU times: user 24.5 ms, sys: 202 µs, total: 24.7 ms
Wall time: 76.7 ms
CPU times: user 14 ms, sys: 3.4 ms, total: 17.4 ms
Wall time: 71.8 ms
using url
presign success
CPU times: user 22.5 ms, sys: 276 µs, total: 22.8 ms
Wall time: 65.5 ms
CPU times: user 5.71 ms, sys: 0 ns, total: 5.71 ms
Wall time: 5.52 ms
CPU times: user 13 ms, sys: 0 ns, total: 13 ms
Wall time: 99.9 ms


In [ ]:
#export
class Requests:
    '''
      for uploading and downloading contents from url
    '''
    @staticmethod
    def getContentFromUrl( url):
      result = requests.get(url)
      if not result.ok:
        print('error downloading')
        return result.content
      content = result.content
      decompressedContent = bz2.decompress(content)
      contentDict = json.loads(decompressedContent)
      return contentDict

In [ ]:
# hide
bucket = 'pybz-test'
key = 'test.dict'
sampleDict = {'test': 'bool'}
USER = None
PW = None

In [ ]:
import pandas as pd
df = pd.DataFrame({'test':[1,2,3,4,5],'test2':[2,3,4,5,6]})
S3.saveDataFrame(bucket,key,df)
S3.loadDataFrame(bucket,key)


,Unnamed: 0,test,test2
0,0,1,2
1,1,2,3
2,2,3,4
3,3,4,5
4,4,5,6


In [ ]:
#hide
!bash build.sh
# !nbdev_build_docs --mk_readme True

build.sh: line 1: nbdev_build_lib: command not found
build.sh: line 2: nbdev_build_docs: command not found
build.sh: line 3: nbdev_clean_nbs: command not found


In [ ]:
%time S3.presign(key='allData', bucket = 'product-bucket-dev-manual')
%time S3.presign(key='allData', bucket = 'product-bucket-dev-manual', checkExist=False)
%time S3.exist(key='allData', bucket = 'product-bucket-dev-manual')

CPU times: user 23.9 ms, sys: 4.01 ms, total: 27.9 ms
Wall time: 118 ms
CPU times: user 2.93 ms, sys: 3.89 ms, total: 6.82 ms
Wall time: 6.84 ms
CPU times: user 17.4 ms, sys: 0 ns, total: 17.4 ms
Wall time: 103 ms


True

In [ ]:
%%time
import boto3
from botocore.errorfactory import ClientError

s3 = boto3.client('s3')
try:
    s3.head_object(Bucket='product-bucket-dev-manual', Key='allData')
except ClientError:
    # Not found
    pass

CPU times: user 17.2 ms, sys: 0 ns, total: 17.2 ms
Wall time: 55.3 ms


In [ ]:
#export
from botocore.exceptions import ClientError
@add_class_method(S3)
def createBucket(cls, bucket:str, **kwargs):
  s3 = cls.s3(**kwargs)
  try:
    s3.create_bucket(
      Bucket=bucket,
      CreateBucketConfiguration={'LocationConstraint':'ap-southeast-1'})
  except ClientError as e:
    print(e)
  response = s3.put_bucket_accelerate_configuration(
      Bucket=bucket ,
      AccelerateConfiguration={
          'Status': 'Enabled'
      }
  )
  return response
@add_class_method(S3)
def deleteBucket(cls, bucket:str, **kwargs):
  s3 = cls.s3(**kwargs)
  response = s3.delete_bucket(
    Bucket=bucket)
  return response

In [ ]:
# bucket= 'tempoprary-test-bucket'
# S3.createBucket(bucket= bucket)
# S3.deleteBucket(bucket= bucket)

In [ ]:
# S3.deleteBucket(bucket= bucket)